In [1]:
### 입력값
filepath_financial = 'financial'

In [2]:
### 처음에만 사용하세요 ###
# !pip install PyQt5

In [16]:
error_code_list = []

In [34]:
#-*-coding: utf-8
# from request_wrapper import wrapped_requests as requests
# from dart_crawler import DartCrawler
from utils import removeEmpty

from selenium.webdriver.common.alert import Alert
from datetime import datetime, timedelta
import xml.etree.ElementTree as ET
from selenium import webdriver
from bs4 import BeautifulSoup
from copy import deepcopy
import configparser
import subprocess
import datetime
import openpyxl
import sqlite3
import pickle
import random
import xlrd
import time
import json
import glob
import sys
import re
import os

class FinancialAgent:
    url = 'https://kind.krx.co.kr/corpgeneral/corpList.do?method=loadInitPage'
    financial_info_keys_kor = ['매출액'
                                ,'영업이익'
                                ,'영업이익(발표기준)'
                                ,'세전계속사업이익'
                                ,'당기순이익'
                                ,'당기순이익(지배)'
                                ,'당기순이익(비지배)'
                                ,'자산총계'
                                ,'부채총계'
                                ,'자본총계'
                                ,'자본총계(지배)'
                                ,'자본총계(비지배)'
                                ,'자본금'
                                ,'영업활동현금흐름'
                                ,'투자활동현금흐름'
                                ,'재무활동현금흐름'
                                ,'CAPEX'
                                ,'FCF'
                                ,'이자발생부채'
                                ,'영업이익률'
                                ,'순이익률'
                                ,'ROE(%)'
                                ,'ROA(%)'
                                ,'부채비율'
                                ,'자본유보율'
                                ,'EPS(원)'
                                ,'PER(배)'
                                ,'BPS(원)'
                                ,'PBR(배)'
                                ,'현금DPS(원)'
                                ,'현금배당수익률'
                                ,'현금배당성향(%)'
                                ,'발행주식수(보통주)'
                            ]
    financial_info_keys_eng = ['sales'
                                ,'op_income'
                                ,'op_income_when_announced'
                                ,'BTCF'
                                ,'net_income'
                                ,'net_income_dom'
                                ,'net_income_non_dom'
                                ,'asset'
                                ,'debt'
                                ,'equity'
                                ,'equity_dom'
                                ,'equity_non_dom'
                                ,'capital_stock'
                                ,'op_CF'
                                ,'inv_CF'
                                ,'fin_CF'
                                ,'CAPEX'
                                ,'FCF'
                                ,'debt_with_interest'
                                ,'op_income_r'
                                ,'income_r'
                                ,'ROE'
                                ,'ROA'
                                ,'debt_r'
                                ,'reserve_r'
                                ,'EPS'
                                ,'PER'
                                ,'BPS'
                                ,'PBR'
                                ,'cash_DPS'
                                ,'cash_dividend_yield'
                                ,'dividend_payout_r'
                                ,'tot_issued_stock'
                            ]
    
    def __init__(self):
        self.save_dir = './data/financial'
        self.save_code_list_path = f'{self.save_dir}/jongmok_code_list.json'
        self.format_file = f'{self.save_dir}/format.xlsx'

        options = webdriver.ChromeOptions()
        options.add_argument('headless')
        self.driver = webdriver.Chrome('./chromedriver/chromedriver', options=options)
        self.driver.implicitly_wait(3)
        
    def downloadJongmokCodeList(self, path = ''): #######수정필요
        if path == '':
            options = webdriver.ChromeOptions()
            self.driver_with_window = webdriver.Chrome('./chromedriver/chromedriver', options=options)
            self.driver_with_window.implicitly_wait(3)

            self.driver_with_window.get(self.url)
            time.sleep(5)
            gotit = self.driver_with_window.find_element_by_xpath('//*[@id="searchForm"]/section/div/div[3]/a[2]')
            self.driver_with_window.execute_script("fnDownload();", gotit)
    #         downloadcsv = self.driver_with_window.find_element_by_css_selector('.icon-csv')
    #         downloadcsv.click()
            time.sleep(5)
            self.driver_with_window.quit()

            abs_path = os.getcwd()
            abs_download_path = '\\'.join(abs_path.split('\\')[0:3]) + '\\Downloads'
            try:
                wb = openpyxl.load_workbook(self._findLatest(abs_download_path))
            except:
                wb = xlrd.open_workbook(self._findLatest(abs_download_path))
            ws = wb['상장법인목록']
        else:
            wb = openpyxl.load_workbook(path)
            ws = wb['상장법인목록']
        output_list = []
        for row in range(2, ws.max_row + 1):
            output_list.append(ws.cell(row=row, column=2).value)
            with open(self.save_code_list_path, "w") as json_file:
                json.dump(output_list, json_file, indent = 4, ensure_ascii=False)
    
    def _getPageSource(self, jongmok_code):
        url = f'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd={jongmok_code}'
        print(f'[getPageSource] url : {url}')
        try:
            time.sleep(1)
            self.driver.get(url)
            self.driver.implicitly_wait(1)
            self.driver.find_element_by_id('cns_td21').click()
            page = self.driver.page_source
            return True, page
        except Exception as e:
            if "올바른 종목이 아닙니다" in e.args[0]:
                print(url, e.args[0])
                error_code_list.append(jongmok_code)
            else:
                error_code_list.append('{} unknownError'.format(jongmok_code))
                return False, ""
        try:
            driver.find_element_by_id('cns_Tab21').click()
        except:
            error_code_list.append('{} unknownError2'.format(jongmok_code))
            return False, ""
        
    def _translator(self, name):
        return self.financial_info_keys_eng[self.financial_info_keys_kor.index(name)]
    
    def _yearMaker(self, string):
        if 'E' in string:
            return string.split('/')[0] + 'E'
        else:
            return string.split('/')[0]
    
    def __saveFinancialExcel(self, jongmok_code): # 사용하지 않는 메소드
        save_name = f'{self.save_dir}/{jongmok_code}.xlsx'

        result, page = self._getPageSource(jongmok_code)
        if result == False:
            return
        
        soup = BeautifulSoup(page, 'html.parser')        
        table_list = soup.select('table.gHead01')
        table_item = None
        for table in table_list:
            caption = removeEmpty(table.select('caption.blind')[0].text)
            if "주요재무정보" == caption:
                table_item = table
                break
        if table_item is None:
            return

        wb = openpyxl.load_workbook(self.format_file)
        ws = wb['Sheet1']
        num_rows = ws.cell(row=1, column=2).value

        jongmok_name = soup.select('span.name')[0].text
        corp_WICS = removeEmpty(soup.select('td.td0101')[0].select('dt.line-left')[2].text.split(':')[1])
        ws.cell(row=2, column=2).value = jongmok_code
        ws.cell(row=3, column=2).value = jongmok_name
        ws.cell(row=4, column=2).value = corp_WICS

        tr_list = table_item.select('thead > tr')
        for tr_idx, tr_item in enumerate(tr_list):
            ws.cell(row=1, column=2).value += 1
            num_rows = ws.cell(row=1, column=2).value
            th_list = tr_item.select('th')
            if tr_idx == 0:
                ws.cell(row=4 + num_rows, column=1).value = removeEmpty(th_list[0].text)
                ws.cell(row=4 + num_rows, column=2).value = removeEmpty(th_list[0].text)
                ws.cell(row=4 + num_rows, column=6).value = removeEmpty(th_list[0].text)
            else:
                for th_idx in range(len(th_list)):
                    ws.cell(row=4 + num_rows, column=2 + th_idx).value = removeEmpty(th_list[th_idx].text)

        tr_list = table_item.select('tbody > tr')
        for tr_idx, tr_item in enumerate(tr_list):
            ws.cell(row=1, column=2).value += 1
            num_rows = ws.cell(row=1, column=2).value
            th_list = tr_item.select('th')
            ws.cell(row=4 + num_rows, column=1).value = removeEmpty(th_list[0].text)    
            td_list = tr_item.select('td')
            for td_idx in range(len(td_list)):
                ws.cell(row=4 + num_rows, column=2 + td_idx).value = removeEmpty(td_list[td_idx].text)
        wb.save(save_name)
        
    def _findLatest(self, path):
        file_name_and_time_list = [] 
        for f_name in os.listdir(f"{path}"):
            written_time = os.path.getctime(f"{path}/{f_name}")
            file_name_and_time_list.append((f_name, written_time))
        sorted_file_list = sorted(file_name_and_time_list, key=lambda x: x[1], reverse=True)
        recent_file = sorted_file_list[0]
        return path + '\\' + recent_file[0]
        
    def _openFinancialList(self):
        self.save_code_list_path
    
    def _updateFinancial(self, jongmok_code, dict_input):
        if 'year_latest' in dict_input.keys():
            if dict_input['year_latest']+1 >= datetime.date.today().year:
                return False, dict_input
        result, page = self._getPageSource(jongmok_code)
        if result == False:
            return False, dict_input
        
        soup = BeautifulSoup(page, 'html.parser')        
        table_list = soup.select('table.gHead01')
        table_item = None
        for table in table_list:
            caption = removeEmpty(table.select('caption.blind')[0].text)
            if "주요재무정보" == caption:
                table_item = table
                break
        if table_item is None:
            return False, dict_input

        jongmok_name = soup.select('span.name')[0].text
        corp_WICS = removeEmpty(soup.select('td.td0101')[0].select('dt.line-left')[2].text.split(':')[1])
        dict_input['jongmok_code'] = jongmok_code
        dict_input['jongmok_name'] = jongmok_name
        dict_input['corp_WICS'] = corp_WICS
        if 'financial_info' not in dict_input.keys():
            dict_input['financial_info'] = {}

        tr_list = table_item.select('thead > tr') # 표의 제목 부분
        year_list = []
        for tr_idx, tr_item in enumerate(tr_list):
            th_list = tr_item.select('th')
            if tr_idx == 0:
                continue
            else :
                for th_idx in range(len(th_list)):
                    year_list.append(self._yearMaker(removeEmpty(th_list[th_idx].text)))
        
        year_latest_updated = False
        is_year = False
        for idx, year in enumerate(year_list):
            if year not in dict_input['financial_info'].keys():
                dict_input['financial_info'][year] = {}
            if year_latest_updated:
                continue
            elif 'E' in year:
                year_latest_updated = True
                year_latest = year_list[idx - 1]
                is_year = True
        
        if 'year_latest' in dict_input.keys():
            if int(year_latest) <= dict_input['year_latest']:
                return False, dict_input
            idx_latest = year_list.index(str(dict_input['year_latest']))
        else:
            idx_latest = -1
            
        if is_year:
            dict_input['year_latest'] = int(year_latest)
        else: dict_input['year_latest'] = -1
        
        tr_list = table_item.select('tbody > tr') # 표의 내용 부분
        th_keylist = []
        for tr_idx, tr_item in enumerate(tr_list):
            th_list = tr_item.select('th') # key값들
            th_keylist.append(self._translator(removeEmpty(th_list[0].text)))
            td_list = tr_item.select('td') # value값들
            for td_idx in range(len(td_list)):
                if td_idx > idx_latest:
                    dict_input['financial_info'][year_list[td_idx]][th_keylist[-1]] = removeEmpty(td_list[td_idx].text) 
        return True, dict_input
        
    def saveFinancial(self, jongmok_code):
        save_name = f'{self.save_dir}/{jongmok_code}.json'
        if os.path.isfile(save_name):
            with open(save_name, "r") as json_file:
                dict_input = json.load(json_file)
            val, dict = self._updateFinancial(jongmok_code, dict_input)
        else:
            val, dict = self._updateFinancial(jongmok_code, {})
        if val:
            with open(save_name, "w") as json_file:
                json.dump(dict, json_file, indent = 4, ensure_ascii=False)
    
    def saveFinancialList(self, post_list):
        for post_idx, post in enumerate(post_list):
            print(f"[saveFinancialList] {post_idx + 1} / {len(post_list)}")
            self.saveFinancial(post)
            print(post)
            print('Log : data saving finished ({0})'.format(post))
            
    def crawl(self):
        with open(self.save_code_list_path, "r") as json_file:
            code_list = json.load(json_file)
        self.saveFinancialList(code_list)
    
    def get(self, jongmok_code):
        save_name = f'{self.save_dir}/{jongmok_code}.json'
        with open(save_name, "r") as json_file:
            json_data = json.load(json_file)
        return json_data

In [26]:
if __name__ == "__main__":
    financial_agent = FinancialAgent()
    financial_agent.crawl()
    print('Log : errored jongmok codes are {}'.format(error_code_list))
#     dart_crawler = DartCrawler()

#     post_list = dart_crawler.getUncheckedPostList()
#     financial_crawler.saveFinancialList(post_list)

[saveFinancialList] 1 / 2450
365270
Log : data saving finished (365270)
[saveFinancialList] 2 / 2450
137310
Log : data saving finished (137310)
[saveFinancialList] 3 / 2450
352910
Log : data saving finished (352910)
[saveFinancialList] 4 / 2450
097520
Log : data saving finished (097520)
[saveFinancialList] 5 / 2450
357580
Log : data saving finished (357580)
[saveFinancialList] 6 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=318660
318660
Log : data saving finished (318660)
[saveFinancialList] 7 / 2450
303530
Log : data saving finished (303530)
[saveFinancialList] 8 / 2450
232680
Log : data saving finished (232680)
[saveFinancialList] 9 / 2450
380320
Log : data saving finished (380320)
[saveFinancialList] 10 / 2450
376190
Log : data saving finished (376190)
[saveFinancialList] 11 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=379390
379390
Log : data saving finished (379390)
[saveFinancial

352700
Log : data saving finished (352700)
[saveFinancialList] 60 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=086710
086710
Log : data saving finished (086710)
[saveFinancialList] 61 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=375500
375500
Log : data saving finished (375500)
[saveFinancialList] 62 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=236810
236810
Log : data saving finished (236810)
[saveFinancialList] 63 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=368770
368770
Log : data saving finished (368770)
[saveFinancialList] 64 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=257990
257990
Log : data saving finished (257990)
[saveFinancialList] 65 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?

363280
Log : data saving finished (363280)
[saveFinancialList] 109 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=318410
318410
Log : data saving finished (318410)
[saveFinancialList] 110 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=355150
355150
Log : data saving finished (355150)
[saveFinancialList] 111 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=347770
347770
Log : data saving finished (347770)
[saveFinancialList] 112 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=291650
291650
Log : data saving finished (291650)
[saveFinancialList] 113 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=294090
294090
Log : data saving finished (294090)
[saveFinancialList] 114 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001

353200
Log : data saving finished (353200)
[saveFinancialList] 158 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=349720
349720
Log : data saving finished (349720)
[saveFinancialList] 159 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=329020
329020
Log : data saving finished (329020)
[saveFinancialList] 160 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=344860
344860
Log : data saving finished (344860)
[saveFinancialList] 161 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=343090
343090
Log : data saving finished (343090)
[saveFinancialList] 162 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=347140
347140
Log : data saving finished (347140)
[saveFinancialList] 163 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001

297090
Log : data saving finished (297090)
[saveFinancialList] 207 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=103840
103840
Log : data saving finished (103840)
[saveFinancialList] 208 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=190650
190650
Log : data saving finished (190650)
[saveFinancialList] 209 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=322000
322000
Log : data saving finished (322000)
[saveFinancialList] 210 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=306040
306040
Log : data saving finished (306040)
[saveFinancialList] 211 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=216080
216080
Log : data saving finished (216080)
[saveFinancialList] 212 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001

...

011000
Log : data saving finished (011000)
[saveFinancialList] 2239 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=009310
009310
Log : data saving finished (009310)
[saveFinancialList] 2240 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=010100
010100
Log : data saving finished (010100)
[saveFinancialList] 2241 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=005070
005070
Log : data saving finished (005070)
[saveFinancialList] 2242 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=004830
004830
Log : data saving finished (004830)
[saveFinancialList] 2243 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=001720
001720
Log : data saving finished (001720)
[saveFinancialList] 2244 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1

005250
Log : data saving finished (005250)
[saveFinancialList] 2287 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=000480
000480
Log : data saving finished (000480)
[saveFinancialList] 2288 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=004310
004310
Log : data saving finished (004310)
[saveFinancialList] 2289 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=003920
003920
Log : data saving finished (003920)
[saveFinancialList] 2290 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=008350
008350
Log : data saving finished (008350)
[saveFinancialList] 2291 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=003350
003350
Log : data saving finished (003350)
[saveFinancialList] 2292 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1

007540
Log : data saving finished (007540)
[saveFinancialList] 2335 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=003280
003280
Log : data saving finished (003280)
[saveFinancialList] 2336 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=001250
001250
Log : data saving finished (001250)
[saveFinancialList] 2337 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=002990
002990
Log : data saving finished (002990)
[saveFinancialList] 2338 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=001470
001470
Log : data saving finished (001470)
[saveFinancialList] 2339 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=001820
001820
Log : data saving finished (001820)
[saveFinancialList] 2340 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1

004540
Log : data saving finished (004540)
[saveFinancialList] 2383 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=001420
001420
Log : data saving finished (001420)
[saveFinancialList] 2384 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=002020
002020
Log : data saving finished (002020)
[saveFinancialList] 2385 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=005930
005930
Log : data saving finished (005930)
[saveFinancialList] 2386 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=000300
000300
Log : data saving finished (000300)
[saveFinancialList] 2387 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=001460
001460
Log : data saving finished (001460)
[saveFinancialList] 2388 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1

003550
Log : data saving finished (003550)
[saveFinancialList] 2431 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=000640
000640
Log : data saving finished (000640)
[saveFinancialList] 2432 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=003690
003690
Log : data saving finished (003690)
[saveFinancialList] 2433 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=000970
000970
Log : data saving finished (000970)
[saveFinancialList] 2434 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=003300
003300
Log : data saving finished (003300)
[saveFinancialList] 2435 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=004250
004250
Log : data saving finished (004250)
[saveFinancialList] 2436 / 2450
[getPageSource] url : https://navercomp.wisereport.co.kr/v2/company/c1